In [184]:
import json
import re
import nltk

In [185]:
# This file contains 8600ish users Data
# In the form: {'MBTI Type', 'Social Media Posts'}
dataFile = open('formatted_data.json', 'r')

In [186]:
data = []

In [187]:
# loading the json data into a list
for line in dataFile:
    data.append(json.loads(line))

In [188]:
types = []
posts = []

In [189]:
# Separating data into 2 separate lists for preprocessing
# For the most part we process the 'post' data
# we skip the first element as that is only the label
for i in range(1, len(data)):
    types.append(data[i]['Type'])
    posts.append(data[i]['Post'])

In [190]:
# List shuffling to make sure our models are valid
# Commented out for debugging purposes
import random
SEED = 673

random.seed(SEED)
random.shuffle(types)
random.shuffle(posts)

In [191]:
# Turning the posts from: 'post1|||post2|||post3'
#                     to: ['post1', 'post2', 'post3']
# expects a list of posts as strings
# returns a list of lists of posts
def vectorize_post_data(posts):
    for index in range(0, len(posts)):
        posts[index] = posts[index].split("|||")
        
    return posts
        
posts = vectorize_post_data(posts)

In [192]:
def remove_hyperTextLinksFromPosts(posts):
    for index in range(0, len(posts)):
        usable_post = [post for post in posts[index] if not re.search(r'^(.)*http(.)*$', post)]
        posts[index] = usable_post
    
    return posts
        
posts = remove_hyperTextLinksFromPosts(posts)

In [193]:
def remove_MBTIClassifiersFromPosts(posts):
    MBTI_regex = r'[\w]*(i|e)(s|n)(f|t)(p|j)[\w]*'
    for i in range(0, len(posts)):
        for j in range(0, len(posts[i])):
            posts[i][j] = re.sub(MBTI_regex, ' ', posts[i][j], flags=re.IGNORECASE)
        
    return posts
    
posts = remove_MBTIClassifiersFromPosts(posts)

In [194]:
from nltk.tokenize import RegexpTokenizer

def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def tokenize_posts(posts):
    tokenizer = RegexpTokenizer(r'\w+')
    
    for i in range(0, len(posts)):
        user_words = []
        for j in range(0, len(posts[i])):
            post = tokenizer.tokenize(posts[i][j])
            for word in post:
                if not isInt(word):
                    user_words.append(word.lower())
        posts[i] = user_words
    
    return posts

posts = tokenize_posts(posts)

In [195]:
all_words_list = []

for user in posts:
    for word in user:
        all_words_list.append(word)


In [196]:
from collections import Counter

In [197]:
freq_list = Counter(all_words_list)
dictionary = freq_list.most_common(10000)

In [198]:
dictionary = list(zip(*dictionary))[0]

In [199]:
nums = range(0, 10000)
word_int = dict(zip(dictionary, nums))

In [200]:
x_vals = []

for user in posts:
    x_vals.append([word_int[x] for x in user if x in word_int.keys()])

In [201]:
intro_extro = [word[0:1] for word in types[0:len(types)]]
bin_intro_extro = []
for letter in intro_extro:
    if (letter == 'I'):
        bin_intro_extro.append(0)
    else:
        bin_intro_extro.append(1)

In [202]:
import numpy as np
x = np.array(x_vals)
test_data = x[:1500]
train_data = x[1500:]

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [203]:
from keras.utils.np_utils import to_categorical

y = np.asarray(bin_intro_extro).astype('float32')
y_test = y[:1500]
y_train = y[1500:]

In [204]:
x_val = x_train[:1500]
x_partial_train = x_train[1500:]

y_val = y_train[:1500]
y_partial_train = y_train[1500:]

In [209]:
# K-Fold Validation
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape = (10000,)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

k = 4

# // is floor div operator
num_val_samples = len(x_train) // k
num_epochs = 13
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
            epochs=num_epochs, batch_size=17, verbose=0)
    
    val_loss, val_acc = model.evaluate(val_data, val_targets)
    print ("Accuracy:", val_acc)
    all_scores.append(val_acc)
    
print ("Accuracy Mean:", np.mean(all_scores))

processing fold # 0
1793/1793 [==============================] - 1s 397us/step
Accuracy: 0.6731734523145566
processing fold # 1
1793/1793 [==============================] - 1s 402us/step
Accuracy: 0.7194645844952593
processing fold # 2
1793/1793 [==============================] - 1s 427us/step
Accuracy: 0.6971556051310652
processing fold # 3
1793/1793 [==============================] - 1s 392us/step
Accuracy: 0.7267150027886224
Accuracy Mean: 0.7041271611823758


In [216]:
model = build_model()
model.fit(x_train, y_train, epochs = 13, batch_size = 512)

Epoch 1/13
7175/7175 [==============================] - 2s 289us/step - loss: 0.5677 - acc: 0.7668
Epoch 2/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.5097 - acc: 0.7711
Epoch 3/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.4875 - acc: 0.7714
Epoch 4/13
7175/7175 [==============================] - 1s 84us/step - loss: 0.4609 - acc: 0.7739
Epoch 5/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.4188 - acc: 0.7815
Epoch 6/13
7175/7175 [==============================] - 1s 84us/step - loss: 0.3975 - acc: 0.7925
Epoch 7/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.3844 - acc: 0.8205
Epoch 8/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.3538 - acc: 0.8368
Epoch 9/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.3423 - acc: 0.8603
Epoch 10/13
7175/7175 [==============================] - 1s 106us/step - loss: 0.3085 - acc: 0.8782
Epoch 11/13
7175/

In [217]:
results = model.evaluate(x_test, y_test)
results

1500/1500 [==============================] - 1s 508us/step


[0.9469588934580485, 0.7573333336512248]